In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from data_fetcher.data_fetcher import DataFetcher
import os, sys, json, time

In [2]:
aws_connection = DataFetcher(
    username = 'admin',
    password = 'energy2021!',
    endpoint = 'database-1.canx610strnv.us-east-1.rds.amazonaws.com',
    database = 'energy',
    port = 3306)
engine,connection,metadata = aws_connection.connect()

dfs = {table:pd.read_sql(f"SELECT * FROM {table}", engine) for table in aws_connection.tables}
for table in dfs.keys():
    exec(f'df_{table} = dfs[table]')

In [3]:
df_demand_dev.head()

,index,date_ts,MWh
0,0,2021-12-11 12:00:00,24996
1,1,2021-12-11 11:00:00,26359
2,2,2021-12-11 10:00:00,26936
3,3,2021-12-11 09:00:00,27929
4,4,2021-12-11 08:00:00,28231


In [4]:
df_generation_dev.head()

,index,hydro_MWh,solar_MWh,wind_MWh
0,2021-12-17 08:00:00,1353,0,1014
1,2021-12-17 07:00:00,1326,0,1197
2,2021-12-17 06:00:00,1666,0,1341
3,2021-12-17 05:00:00,1890,0,1976
4,2021-12-17 04:00:00,2198,0,2346


In [5]:
    df_delta = (df_generation_dev
                .merge(df_demand_dev, left_on = 'index', right_on ='date_ts')
                .drop(columns = ['index_y','date_ts'])
                .rename(columns = {'index_x':'index','MWh':'demand_MWh'})
                .assign(balance = lambda x: x.hydro_MWh + x.solar_MWh + x.wind_MWh - x.demand_MWh)
                .drop(columns = ['hydro_MWh','solar_MWh','wind_MWh','demand_MWh'])
                )
    
    df_delta.head()

,index,balance
0,2021-12-11 12:00:00,-23499
1,2021-12-11 11:00:00,-24698
2,2021-12-11 10:00:00,-25315
3,2021-12-11 09:00:00,-26223
4,2021-12-11 08:00:00,-26343


In [6]:
dfs.keys()

dict_keys(['demand_dev', 'generation_dev', 'hydro_dev', 'solar_dev', 'wind_dev'])

In [7]:
dfs['hydro_dev']

,index,coord_lon,coord_lat,weather_0_id,weather_0_main,weather_0_description,weather_0_icon,base,main_temp,main_feels_like,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod
0,0,-119.5779,37.7407,802,Clouds,scattered clouds,03d,stations,-1.26,-5.76,...,1639842322,2,2013018,US,1639840115,1639874502,-28800,7262586,Yosemite Valley,200
1,1,-120.0510,39.2399,803,Clouds,broken clouds,04d,stations,-6.68,-11.16,...,1639842322,2,2004297,US,1639840491,1639874353,-28800,5400943,Tahoe Vista,200
2,2,-120.5844,40.6666,804,Clouds,overcast clouds,04d,stations,-9.92,-9.92,...,1639842322,1,5946,US,1639840882,1639874219,-28800,5566544,Lassen,200
3,3,-108.5016,59.5505,804,Clouds,overcast clouds,04n,stations,-26.57,-33.57,...,1639842322,1,197,CA,1639843808,1639865490,-21600,5947462,Eldorado,200


In [8]:
(df_generation_dev
          #.set_index('index')
          .assign(total = lambda x: x.hydro_MWh + x.solar_MWh + x.wind_MWh)
          .rename(columns={'total':'Generation'})
          )

,index,hydro_MWh,solar_MWh,wind_MWh,Generation
0,2021-12-17 08:00:00,1353,0,1014,2367
1,2021-12-17 07:00:00,1326,0,1197,2523
2,2021-12-17 06:00:00,1666,0,1341,3007
3,2021-12-17 05:00:00,1890,0,1976,3866
4,2021-12-17 04:00:00,2198,0,2346,4544
...,...,...,...,...,...
1845,2021-10-01 11:00:00,1238,0,244,1482
1846,2021-10-01 10:00:00,1352,0,240,1592
1847,2021-10-01 09:00:00,1495,0,291,1786
1848,2021-10-01 08:00:00,1399,0,573,1972


In [9]:
import altair as alt
from vega_datasets import data

alt.data_transformers.disable_max_rows()
# source = (df_generation_dev
#           #.set_index('index')
#           .assign(total = lambda x: x.hydro_MWh + x.solar_MWh + x.wind_MWh)
#           .rename(columns={'total':'Generation'})
#           )

source = pd.concat(
        [
            df_generation_dev[['index','hydro_MWh']].rename(columns={'hydro_MWh':'generation'}).assign(type='Hydro'),
            df_generation_dev[['index','solar_MWh']].rename(columns={'solar_MWh':'generation'}).assign(type='Solar'),
            df_generation_dev[['index','wind_MWh']].rename(columns={'wind_MWh':'generation'}).assign(type='Wind'),
        ]
    )

alt.Chart(source).mark_area().encode(
    x="index:T",
    y=alt.Y("generation:Q", stack="normalize"),
    color="type:N"
)

alt.Chart(...)

In [10]:
dfs.keys()
#df_generation_sorted = pd.concat[df_generation_dev[['hydro_MWh','index']],

dict_keys(['demand_dev', 'generation_dev', 'hydro_dev', 'solar_dev', 'wind_dev'])

In [12]:
df_test2 = pd.concat(
            [
                df_generation_dev[['index','hydro_MWh']].rename(columns={'hydro_MWh':'Generation','index':'Date'}).assign(Type='Hydro'),
                df_generation_dev[['index','solar_MWh']].rename(columns={'solar_MWh':'Generation','index':'Date'}).assign(Type='Solar'),
                df_generation_dev[['index','wind_MWh']].rename(columns={'wind_MWh':'Generation','index':'Date'}).assign(Type='Wind'),
                df_generation_dev.assign(total = lambda x: x.hydro_MWh + x.solar_MWh + x.wind_MWh).rename(columns={'total':'Generation','index':'Date'}).drop(columns = ['hydro_MWh','solar_MWh','wind_MWh']).assign(Type='Total'),
            ]
        )

df_test2.head()

alt.Chart(df_test2).mark_bar().encode(
    x='Date',
    y='sum(Generation)',
    color='Type'
)

alt.Chart(...)

In [13]:
source = data.barley()

alt.Chart(source).mark_bar().encode(
    x='variety',
    y='sum(yield)',
    color='site'
)

alt.Chart(...)

In [14]:
source.head()

,yield,variety,year,site
0,27.00000,Manchuria,1931,University Farm
1,48.86667,Manchuria,1931,Waseca
2,27.43334,Manchuria,1931,Morris
3,39.93333,Manchuria,1931,Crookston
4,32.96667,Manchuria,1931,Grand Rapids


In [15]:
dfs.keys()

dict_keys(['demand_dev', 'generation_dev', 'hydro_dev', 'solar_dev', 'wind_dev'])

In [23]:
dfs['hydro_dev'][['coord_lon','coord_lat']].head()

,coord_lon,coord_lat
0,-119.5779,37.7407
1,-120.0510,39.2399
2,-120.5844,40.6666
3,-108.5016,59.5505


In [16]:
dfs['hydro_dev'].head()

,index,coord_lon,coord_lat,weather_0_id,weather_0_main,weather_0_description,weather_0_icon,base,main_temp,main_feels_like,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod
0,0,-119.5779,37.7407,802,Clouds,scattered clouds,03d,stations,-1.26,-5.76,...,1639842322,2,2013018,US,1639840115,1639874502,-28800,7262586,Yosemite Valley,200
1,1,-120.0510,39.2399,803,Clouds,broken clouds,04d,stations,-6.68,-11.16,...,1639842322,2,2004297,US,1639840491,1639874353,-28800,5400943,Tahoe Vista,200
2,2,-120.5844,40.6666,804,Clouds,overcast clouds,04d,stations,-9.92,-9.92,...,1639842322,1,5946,US,1639840882,1639874219,-28800,5566544,Lassen,200
3,3,-108.5016,59.5505,804,Clouds,overcast clouds,04n,stations,-26.57,-33.57,...,1639842322,1,197,CA,1639843808,1639865490,-21600,5947462,Eldorado,200


In [19]:
dfs['hydro_dev'].columns

Index(['index', 'coord_lon', 'coord_lat', 'weather_0_id', 'weather_0_main',
       'weather_0_description', 'weather_0_icon', 'base', 'main_temp',
       'main_feels_like', 'main_temp_min', 'main_temp_max', 'main_pressure',
       'main_humidity', 'main_sea_level', 'main_grnd_level', 'visibility',
       'wind_speed', 'wind_deg', 'wind_gust', 'clouds_all', 'dt', 'sys_type',
       'sys_id', 'sys_country', 'sys_sunrise', 'sys_sunset', 'timezone', 'id',
       'name', 'cod'],
      dtype='object')

In [17]:
dfs['solar_dev'].head()

,index,coord_lon,coord_lat,weather_0_id,weather_0_main,weather_0_description,weather_0_icon,base,main_temp,main_feels_like,...,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod,main_sea_level,main_grnd_level,wind_gust
0,0,-115.4989,32.6790,800,Clear,clear sky,01d,stations,0.90,-1.48,...,US,1639838330,1639874329,-28800,5332698,Calexico,200,NaN,NaN,NaN
1,1,125.6081,9.0125,804,Clouds,overcast clouds,04n,stations,24.11,25.08,...,PH,1639777568,1639819321,28800,1705545,Los Angeles,200,1012.0,1010.0,2.94
2,2,-119.0187,35.3733,701,Mist,mist,50d,stations,7.16,7.16,...,US,1639839590,1639874759,-28800,5325738,Bakersfield,200,NaN,NaN,1.79
3,3,-117.6031,33.6409,800,Clear,clear sky,01d,stations,10.50,8.31,...,US,1639838980,1639874689,-28800,5386082,Rancho Santa Margarita,200,NaN,NaN,1.79
4,4,-121.2811,38.7071,741,Fog,fog,50d,stations,2.07,0.55,...,US,1639840692,1639874743,-28800,5337561,Citrus Heights,200,NaN,NaN,NaN


In [20]:
dfs['solar_dev'].columns

Index(['index', 'coord_lon', 'coord_lat', 'weather_0_id', 'weather_0_main',
       'weather_0_description', 'weather_0_icon', 'base', 'main_temp',
       'main_feels_like', 'main_temp_min', 'main_temp_max', 'main_pressure',
       'main_humidity', 'visibility', 'wind_speed', 'wind_deg', 'clouds_all',
       'dt', 'sys_type', 'sys_id', 'sys_country', 'sys_sunrise', 'sys_sunset',
       'timezone', 'id', 'name', 'cod', 'main_sea_level', 'main_grnd_level',
       'wind_gust'],
      dtype='object')

In [18]:
dfs['wind_dev'].head()

,index,coord_lon,coord_lat,weather_0_id,weather_0_main,weather_0_description,weather_0_icon,base,main_temp,main_feels_like,...,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod,wind_gust
0,0,-118.1740,35.0525,800,Clear,clear sky,01d,stations,-0.17,-4.20,...,1,4892,US,1639839336,1639874607,-28800,5373965,Mojave,200,NaN
1,1,-118.4490,35.1322,800,Clear,clear sky,01d,stations,0.00,0.00,...,2,2036659,US,1639839415,1639874660,-28800,5401297,Tehachapi,200,0.45
2,2,-116.5453,33.8303,800,Clear,clear sky,01d,stations,10.94,8.95,...,1,5412,US,1639838755,1639874406,-28800,5380668,Palm Springs,200,NaN
3,3,-121.7680,37.6819,701,Mist,mist,50d,stations,2.72,2.72,...,2,65634,US,1639840631,1639875038,-28800,5367440,Livermore,200,2.68
4,4,-122.0400,38.2494,701,Mist,mist,50d,stations,3.79,-0.25,...,1,5939,US,1639840794,1639875005,-28800,5347335,Fairfield,200,NaN


In [21]:
dfs['wind_dev'].columns

Index(['index', 'coord_lon', 'coord_lat', 'weather_0_id', 'weather_0_main',
       'weather_0_description', 'weather_0_icon', 'base', 'main_temp',
       'main_feels_like', 'main_temp_min', 'main_temp_max', 'main_pressure',
       'main_humidity', 'visibility', 'wind_speed', 'wind_deg', 'clouds_all',
       'dt', 'sys_type', 'sys_id', 'sys_country', 'sys_sunrise', 'sys_sunset',
       'timezone', 'id', 'name', 'cod', 'wind_gust'],
      dtype='object')